# INSTALL: packages

In [14]:
pip install nbformat

You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [15]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [16]:
import pandas as pd
import os
import pickle

# EXECUTE: WalletClustering_neo4jConnect notebook

In [17]:
%run ./WalletClustering_neo4jConnect.ipynb # includes Neo4J connector
# methods & variables of notebook can be referenced

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# DEFINE: other methods

Were used in older versions of this code. Kept for consistency of other notebooks.

In [18]:
# flattens list of lists
def flatten(list):
    return [item for sublist in list for item in sublist]

# DEFINE: get interesting addresses excluding blacklisted addresses

## create output directory

In [19]:
try:
    os.mkdir('output')
except:
    pass

## query terror addresses

In [20]:
#The below code is returning a list of all the addresses that are marked as terror addresses.
#Query takes around 10 min
def returnTerrorAddresses():

    query_string = '''
    MATCH (a:Address {isTerror: True})
    Return a.address
    '''

    response = conn.query(query_string, db='neo4j')
    terrorAddresses = [r[0] for r in response]
    return terrorAddresses

In [21]:
# comment out entire block if old terrorAddressList saved as pickle should be used
def createTerrorAddressList():
    terrorAddressList = returnTerrorAddresses()
  
    # save terrorAddressList to file
    with open('output\\terrorAddressList.pickle', 'wb') as export:
        pickle.dump(terrorAddressList, export)

## create blacklist from json

In [22]:
# allows blacklist of addresses in json format
def createBlacklist():
  blacklistAddresses = pd.DataFrame(columns =['address', 'association'])
  
  dirname = os.path.dirname(os.path.realpath('__file__'))
  
  # iterate over collection of exchange addresses in json format to create blacklist
  # if os.path.isdir('..\\WalletExplorerScraper\\Output\\exchange\\'):
  #   outputScrapeExchangeFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\exchange\\')
  #   for subfolder in os.listdir(outputScrapeExchangeFolder):
  #       df = pd.read_json(os.path.join(outputScrapeExchangeFolder, subfolder, 'scraped_addresses.json'))
  #       df.columns=['address']
  #       df['association'] = subfolder.split("_",1)[1]
  #       blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
  #       continue
  
  if os.path.isdir('..\\WalletExplorerScraper\\Output\\mixer\\'):
    outputScrapeMixerFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\mixer\\')
    for subfolder in os.listdir(outputScrapeMixerFolder):
        df = pd.read_json(os.path.join(outputScrapeMixerFolder, subfolder, 'scraped_addresses.json'))
        df.columns=['address']
        df['association'] = subfolder.split("_",1)[1]
        blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
        continue

  if os.path.isdir('..\\WalletExplorerScraper\\Output\\giant_wallet\\'):
    outputScrapeGiantWalletFolder = os.path.join(dirname, '..\\WalletExplorerScraper\\Output\\giant_wallet\\')
    for subfolder in os.listdir(outputScrapeGiantWalletFolder):
        df = pd.read_json(os.path.join(outputScrapeGiantWalletFolder, subfolder, 'scraped_addresses.json'))
        df.columns=['address']
        df['association'] = subfolder.split("_",1)[1]
        blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
        continue

  #export blacklistAddresses to file
  with open('output\\blacklistAddresses.pickle', 'wb') as export:
    pickle.dump(blacklistAddresses, export)

## remove blacklisted addresses from addresses of interest

In [23]:
# to be used for removal of exchange addresses
def addressCleanUp(addressList, blacklist):
    interestingAddresses = []
    for address in addressList:
        if not blacklist['address'].str.contains(address).any():
            interestingAddresses.append(address)
    return interestingAddresses

In [24]:
def createAddressesToClusterList():
    addressesToCluster = addressCleanUp(terrorAddressList, blacklistAddresses)

    # save addressesToCluster to file
    with open('output\\addressesToCluster.pickle', 'wb') as export:
        pickle.dump(addressesToCluster, export)

# EXECUTE: get interesting addresses excluding blacklisted addresses

In [25]:
# use existing terrorAddressList if exists
if not os.path.exists('output\\terrorAddressList.pickle'):
    createTerrorAddressList()

terrorAddressList = pickle.load(open('output\\terrorAddressList.pickle', 'rb'))

In [26]:
# use existing blacklist if exists
if not os.path.exists('output\\blacklistAddresses.pickle'):
    createBlacklist()

blacklistAddresses = pickle.load(open('output\\blacklistAddresses.pickle', 'rb'))

True


C:\Users\info\AppData\Local\Temp\ipykernel_23256\68541919.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
C:\Users\info\AppData\Local\Temp\ipykernel_23256\68541919.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
C:\Users\info\AppData\Local\Temp\ipykernel_23256\68541919.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  blacklistAddresses = blacklistAddresses.append(df, ignore_index=True)
C:\Users\info\AppData\Local\Temp\ipykernel_23256\68541919.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

False


In [27]:
# use existing addressesToCluster if exists
if not os.path.exists('output\\addressesToCluster.pickle'):
    createAddressesToClusterList()

addressesToCluster = pickle.load(open('output\\addressesToCluster.pickle', 'rb'))